In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
from bs4 import BeautifulSoup
import re
import threading
import time
import json
import ollama
from datetime import datetime

class ITTicketAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Analizador de Tickets IT")
        self.root.geometry("600x400")
        self.root.resizable(False, False)
        
        self.file_path = ""
        self.processing = False
        self.total_tickets = 0
        self.processed_tickets = 0
        self.errors = 0
        
        self.setup_ui()
        
    def setup_ui(self):
        # Frame superior - Selección de archivo
        top_frame = ttk.LabelFrame(self.root, text="Entrada de datos", padding=10)
        top_frame.pack(fill=tk.X, padx=10, pady=5)
        
        ttk.Label(top_frame, text="Seleccionar archivo Excel:").grid(row=0, column=0, sticky=tk.W)
        
        self.file_entry = ttk.Entry(top_frame, state='readonly', width=50)
        self.file_entry.grid(row=0, column=1, padx=5)
        
        browse_icon = "🖿"  # Icono de examinar
        ttk.Button(top_frame, text=browse_icon, command=self.browse_file, width=3).grid(row=0, column=2)
        
        # Frame central - Logs y progreso
        center_frame = ttk.LabelFrame(self.root, text="Progreso", padding=10)
        center_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        self.log_text = tk.Text(center_frame, height=15, state='disabled')
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        scrollbar = ttk.Scrollbar(center_frame, command=self.log_text.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.config(yscrollcommand=scrollbar.set)
        
        self.progress = ttk.Progressbar(center_frame, mode='determinate')
        self.progress.pack(fill=tk.X, pady=5)
        
        # Frame inferior - Botón de ejecución
        bottom_frame = ttk.Frame(self.root)
        bottom_frame.pack(fill=tk.X, padx=10, pady=5)
        
        run_icon = "▶"  # Icono de ejecutar
        self.run_button = ttk.Button(bottom_frame, text=run_icon, command=self.start_processing)
        self.run_button.pack(pady=5)
        
    def browse_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            self.file_path = file_path
            self.file_entry.config(state='normal')
            self.file_entry.delete(0, tk.END)
            self.file_entry.insert(0, file_path)
            self.file_entry.config(state='readonly')
            self.log_message(f"Archivo seleccionado: {file_path}")
    
    def log_message(self, message):
        self.log_text.config(state='normal')
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.log_text.insert(tk.END, f"[{timestamp}] {message}\n")
        self.log_text.see(tk.END)
        self.log_text.config(state='disabled')
    
    def start_processing(self):
        if not self.file_path:
            messagebox.showerror("Error", "Por favor seleccione un archivo Excel.")
            return
            
        if self.processing:
            messagebox.showwarning("Advertencia", "El proceso ya está en ejecución.")
            return
            
        self.processing = True
        self.run_button.config(state='disabled')
        self.log_message("Iniciando procesamiento de tickets...")
        
        # Iniciar procesamiento en un hilo separado
        processing_thread = threading.Thread(target=self.process_tickets)
        processing_thread.start()
    
    def process_tickets(self):
        start_time = time.time()
        
        try:
            # Leer el archivo Excel
            df = pd.read_excel(self.file_path)
            self.total_tickets = len(df)
            self.progress["maximum"] = self.total_tickets
            self.processed_tickets = 0
            self.errors = 0
            
            # Validar columnas requeridas
            required_columns = [
                'Number', 'Priority', 'State', 'Assigned to', 
                'Short description', 'Task type', 'Subcategory', 
                'Closed', 'Created', 'Assignment group', 
                'Work notes list', 'Work notes', 'Description'
            ]
            
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                self.log_message(f"Error: Faltan columnas requeridas: {', '.join(missing_columns)}")
                messagebox.showerror("Error", f"El archivo no contiene todas las columnas requeridas. Faltan: {', '.join(missing_columns)}")
                return
                
            # Procesar cada ticket
            for idx, row in df.iterrows():
                if not self.processing:  # Permitir cancelación
                    break
                    
                try:
                    # Limpieza de texto
                    description = self.clean_text(row['Description'])
                    work_notes = self.clean_text(row['Work notes'])
                    
                    # Clasificación con LLM
                    summary = self.get_gemma3_summary(description, work_notes)
                    
                    # Agregar nuevas columnas
                    df.at[idx, 'DescripcionMistral'] = "Clasificación detallada"  # Placeholder
                    df.at[idx, 'ResumenGemma3'] = summary
                    
                    self.processed_tickets += 1
                    if self.processed_tickets % 5 == 0:
                        self.log_message(f"Procesados {self.processed_tickets}/{self.total_tickets} tickets")
                        self.progress["value"] = self.processed_tickets
                        self.root.update()
                        
                except Exception as e:
                    self.errors += 1
                    self.log_message(f"Error procesando ticket {row['Number']}: {str(e)}")
                    continue
                    
            # Guardar resultados
            if self.processing:  # Solo si no fue cancelado
                save_path = filedialog.asksaveasfilename(
                    defaultextension=".xlsx",
                    filetypes=[("Excel files", "*.xlsx")],
                    title="Guardar resultados"
                )
                
                if save_path:
                    df.to_excel(save_path, index=False)
                    self.log_message(f"Resultados guardados en: {save_path}")
                    
                    # Estadísticas finales
                    elapsed_time = time.time() - start_time
                    self.log_message("\n--- Estadísticas Finales ---")
                    self.log_message(f"Tiempo total: {elapsed_time:.2f} segundos")
                    self.log_message(f"Tickets procesados: {self.processed_tickets}/{self.total_tickets}")
                    self.log_message(f"Errores: {self.errors}")
                    self.log_message("Proceso completado.")
                    
        except Exception as e:
            self.log_message(f"Error grave: {str(e)}")
            messagebox.showerror("Error", f"Ocurrió un error durante el procesamiento: {str(e)}")
            
        finally:
            self.processing = False
            self.run_button.config(state='normal')
            self.progress["value"] = 0
            self.root.update()
    
    def clean_text(self, text):
        if pd.isna(text):
            return ""
            
        # Eliminar patrones [ARGONAUTA] y [INC...]
        text = re.sub(r'\[ARGONAUTA\]|\[INC[^\]]*\]', '', str(text))
        
        # Eliminar HTML
        soup = BeautifulSoup(text, 'html.parser')
        text = soup.get_text(separator=' ')
        
        # Limpieza adicional
        text = ' '.join(text.split())  # Eliminar espacios múltiples
        
        return text.strip()
    
    def get_gemma3_summary(self, description, work_notes):
        try:
            # Combinar descripción y notas de trabajo
            combined_text = f"Descripción del ticket:\n{description}\n\nNotas de trabajo:\n{work_notes}"
            
            # Crear prompt para Gemma3
            prompt = f"""
            Resume este ticket para análisis de tendencias:
            - Puntos clave técnicos
            - Identificar posible causa raíz
            - Formato: Viñetas concisas
            
            Ticket:
            {combined_text}
            """
            
            # Llamar a Ollama con Gemma3
            response = ollama.generate(
                model='gemma3',
                prompt=prompt,
                format='json',
                options={'temperature': 0.3}
            )
            
            # Procesar respuesta
            try:
                result = json.loads(response['response'])
                summary = "\n".join([f"• {item}" for item in result.get('summary', [])])
                return summary
            except json.JSONDecodeError:
                return response['response']
                
        except Exception as e:
            self.log_message(f"Error con Ollama: {str(e)}")
            return f"Error al generar resumen: {str(e)}"
    
    def on_closing(self):
        if self.processing:
            if messagebox.askokcancel("Salir", "El proceso está en ejecución. ¿Desea cancelar y salir?"):
                self.processing = False
                self.root.destroy()
        else:
            self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ITTicketAnalyzer(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()